In [2]:
import pandas as pd
import numpy as np
import folium

In [3]:
# Download the database titled 'Víctimas en carpetas de investigación (acumulado)' from https://datos.cdmx.gob.mx/dataset/victimas-en-carpetas-de-investigacion-fgj and save it in the 'data' directory
# This database contains data on individuals victimized by felonies in Mexico City, as reported by the Mexico City Prosecutor's Office

data_FGJ = pd.read_csv("data/victimasFGJ.csv", encoding="UTF-8")
data_FGJ.head(5)

,anio_inicio,mes_inicio,fecha_inicio,hora_inicio,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,delito,categoria_delito,...,tipo_persona,calidad_juridica,competencia,colonia_hecho,colonia_catalogo,alcaldia_hecho,alcaldia_catalogo,municipio_hecho,latitud,longitud
0,2019.0,Enero,2019-01-04,12:19:00,2018.0,Agosto,2018-08-29,12:00:00,FRAUDE,DELITO DE BAJO IMPACTO,...,FISICA,OFENDIDO,FUERO COMUN,GUADALUPE INN,Guadalupe Inn,ALVARO OBREGON,Álvaro Obregón,CDMX,19.36125,-99.18314
1,2019.0,Enero,2019-01-04,12:20:00,2018.0,Diciembre,2018-12-15,15:00:00,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,...,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,VICTORIA DE LAS DEMOCRACIAS,Victoria De Las Democracias,AZCAPOTZALCO,Azcapotzalco,CDMX,19.47181,-99.16458
2,2019.0,Enero,2019-01-04,12:23:00,2018.0,Diciembre,2018-12-22,15:30:00,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,...,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,COPILCO UNIVERSIDAD ISSSTE,Copilco El Bajo,COYOACAN,Coyoacán,CDMX,19.33797,-99.18611
3,2019.0,Enero,2019-01-04,12:27:00,2019.0,Enero,2019-01-04,06:00:00,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,...,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,AGRÍCOLA PANTITLAN,Agricola Pantitlan,IZTACALCO,Iztacalco,CDMX,19.40327,-99.05983
4,2019.0,Enero,2019-01-04,12:35:00,2019.0,Enero,2019-01-03,20:00:00,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,...,FISICA,VICTIMA,FUERO COMUN,PROGRESISTA,Progresista,IZTAPALAPA,Iztapalapa,CDMX,19.35480,-99.06324


In [4]:
# Well, let's see the columns and data types

data_FGJ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415763 entries, 0 to 1415762
Data columns (total 22 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   anio_inicio        1415748 non-null  float64
 1   mes_inicio         1415748 non-null  object 
 2   fecha_inicio       1415748 non-null  object 
 3   hora_inicio        1415748 non-null  object 
 4   anio_hecho         1415343 non-null  float64
 5   mes_hecho          1415343 non-null  object 
 6   fecha_hecho        1415342 non-null  object 
 7   hora_hecho         1415353 non-null  object 
 8   delito             1415749 non-null  object 
 9   categoria_delito   1415749 non-null  object 
 10  sexo               1168059 non-null  object 
 11  edad               931053 non-null   float64
 12  tipo_persona       1408196 non-null  object 
 13  calidad_juridica   1415748 non-null  object 
 14  competencia        1415749 non-null  object 
 15  colonia_hecho      1340796 non-n

In [5]:
# Due to Jupyter’s limitations, we need to create a .txt file containing a list of felonies

delitos_unicos = sorted(data_FGJ["delito"].dropna().unique())

with open("delitos_unicos.txt", "w", encoding="utf-8") as f:
    for delito in delitos_unicos:
        f.write(delito + "\n")

In [6]:
# We're only interested in missing people, so we filter it out

missing_cdmx = data_FGJ[data_FGJ["delito"] == "DESAPARICION FORZADA DE PERSONAS"].reset_index(drop=True)

del data_FGJ

In [7]:
# Ok, we need to convert the columns "anio_inicio", "anio_hecho", and "edad" to integers

missing_cdmx[["anio_inicio", "anio_hecho", "edad"]] = missing_cdmx[["anio_inicio", "anio_hecho", "edad"]].apply(pd.to_numeric, errors="coerce").astype("Int64")

# And, we need to convert the columns "fecha_inicio" and "fecha_hecho" to datetime

missing_cdmx["fecha_inicio"] = pd.to_datetime(missing_cdmx["fecha_inicio"], errors="coerce")
missing_cdmx["hora inicio"] = pd.to_datetime(missing_cdmx["hora_inicio"], format="%H:%M:%S", errors="coerce").dt.time

In [8]:
# Let's see the columns

missing_cdmx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   anio_inicio        98 non-null     Int64         
 1   mes_inicio         98 non-null     object        
 2   fecha_inicio       98 non-null     datetime64[ns]
 3   hora_inicio        98 non-null     object        
 4   anio_hecho         98 non-null     Int64         
 5   mes_hecho          98 non-null     object        
 6   fecha_hecho        98 non-null     object        
 7   hora_hecho         98 non-null     object        
 8   delito             98 non-null     object        
 9   categoria_delito   98 non-null     object        
 10  sexo               94 non-null     object        
 11  edad               64 non-null     Int64         
 12  tipo_persona       96 non-null     object        
 13  calidad_juridica   98 non-null     object        
 14  competencia 

In [9]:
# And let's see the head

missing_cdmx.head()

,anio_inicio,mes_inicio,fecha_inicio,hora_inicio,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,delito,categoria_delito,...,calidad_juridica,competencia,colonia_hecho,colonia_catalogo,alcaldia_hecho,alcaldia_catalogo,municipio_hecho,latitud,longitud,hora inicio
0,2019,Mayo,2019-05-06,22:38:00,2019,Mayo,2019-05-05,23:00:00,DESAPARICION FORZADA DE PERSONAS,DELITO DE BAJO IMPACTO,...,VICTIMA,FUERO COMUN,ESCUADRÓN 201,Granjas De San Antonio,IZTAPALAPA,Iztapalapa,CDMX,19.365476,-99.109144,22:38:00
1,2019,Mayo,2019-05-28,02:13:00,2019,Mayo,2019-05-27,23:00:00,DESAPARICION FORZADA DE PERSONAS,DELITO DE BAJO IMPACTO,...,VICTIMA Y DENUNCIANTE,FUERO COMUN,SAN MARTÍN XOCHINAHUAC,Pueblo San Martin Xochinahuac,AZCAPOTZALCO,Azcapotzalco,CDMX,19.497923,-99.192560,02:13:00
2,2019,Septiembre,2019-09-01,04:23:26,2019,Agosto,2019-08-28,02:30:00,DESAPARICION FORZADA DE PERSONAS,DELITO DE BAJO IMPACTO,...,VICTIMA,FUERO COMUN,ÁLAMOS,Moderna,BENITO JUAREZ,Benito Juárez,CDMX,19.395741,-99.137627,04:23:26
3,2019,Septiembre,2019-09-25,11:46:44,2019,Junio,2019-06-27,05:30:00,DESAPARICION FORZADA DE PERSONAS,DELITO DE BAJO IMPACTO,...,VICTIMA,FUERO COMUN,SAN ANGEL,Pueblo Tizapan,ALVARO OBREGON,Álvaro Obregón,CDMX,19.340893,-99.195804,11:46:44
4,2019,Marzo,2019-03-06,12:17:00,2019,Febrero,2019-02-19,10:00:00,DESAPARICION FORZADA DE PERSONAS,DELITO DE BAJO IMPACTO,...,VICTIMA,FUERO COMUN,MORELOS,Morelos,CUAUHTEMOC,Cuauhtémoc,CDMX,19.448231,-99.131688,12:17:00


In [10]:
# And some stats

missing_cdmx.describe()

,anio_inicio,fecha_inicio,anio_hecho,edad,latitud,longitud
count,98.0,98,98.0,64.0,85.000000,85.000000
mean,2021.285714,2021-10-22 00:14:41.632653056,2020.357143,41.03125,19.386121,-99.138589
min,2019.0,2019-02-20 00:00:00,2011.0,15.0,19.215253,-99.273856
25%,2020.0,2020-08-29 12:00:00,2019.0,26.75,19.340893,-99.192560
50%,2021.0,2021-10-19 00:00:00,2020.5,36.0,19.395741,-99.139842
75%,2023.0,2023-04-01 06:00:00,2022.0,51.5,19.439854,-99.100715
max,2024.0,2024-09-23 00:00:00,2024.0,96.0,19.519162,-98.997700
std,1.605403,NaN,2.174121,19.679148,0.070804,0.067768


In [ ]:
# And finally , we# And finally, we need to center the map around the average coordinates of the missing persons data,
# create a Folium map, and plot each individual case as a red circle marker using their latitude and longitude.
# Each marker includes a popup with the date and the municipality (alcaldía) of the disappearance.

lat_centro = missing_cdmx["latitud"].mean()
lon_centro = missing_cdmx["longitud"].mean()

mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=11, tiles="CartoDB Positron")

for _, row in missing_cdmx.dropna(subset=["latitud", "longitud"]).iterrows():
    folium.CircleMarker(
        location=[row["latitud"], row["longitud"]],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        popup=f'Fecha: {row["fecha_inicio"]}<br>Alcaldía: {row["alcaldia_hecho"]}'
    ).add_to(mapa)

mapa